In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Load the data and preprocess it
df = pd.read_csv('/content/drive/MyDrive/Thesis New/fer2013.csv')

# Remove disgust and surprise emotions
df = df[(df['emotion'] != 1) & (df['emotion'] != 5)]

pixels = df['pixels'].tolist()
X = np.array([np.fromstring(pixel, dtype=int, sep=' ') for pixel in pixels])
X = X.reshape(-1, 48, 48, 1) / 255.0  # Normalize pixel values
y = df['emotion'].values

# Perform one-hot encoding
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = pd.get_dummies(y).values  # Convert to one-hot encoding

# Ensure the correct number of output units in the model
num_classes = y.shape[1]  # Get the number of classes from one-hot encoding

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))  # Output layer with num_classes units

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=128, epochs=25, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy on test set: {accuracy * 100:.2f}%')

# Save the model architecture as a JSON file
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

# Save the model weights as an HDF5 file
model.save_weights('model.h5')


Epoch 1/25
164/164 [==============================] - 9s 21ms/step - loss: 1.3501 - accuracy: 0.3477 - val_loss: 1.3255 - val_accuracy: 0.3701
Epoch 2/25
164/164 [==============================] - 2s 12ms/step - loss: 1.2865 - accuracy: 0.4147 - val_loss: 1.2182 - val_accuracy: 0.4701
Epoch 3/25
164/164 [==============================] - 2s 11ms/step - loss: 1.2178 - accuracy: 0.4685 - val_loss: 1.1617 - val_accuracy: 0.5137
Epoch 4/25
164/164 [==============================] - 2s 11ms/step - loss: 1.1692 - accuracy: 0.4935 - val_loss: 1.1174 - val_accuracy: 0.5280
Epoch 5/25
164/164 [==============================] - 2s 11ms/step - loss: 1.1215 - accuracy: 0.5237 - val_loss: 1.0895 - val_accuracy: 0.5347
Epoch 6/25
164/164 [==============================] - 2s 12ms/step - loss: 1.0897 - accuracy: 0.5339 - val_loss: 1.0563 - val_accuracy: 0.5563
Epoch 7/25
164/164 [==============================] - 2s 11ms/step - loss: 1.0616 - accuracy: 0.5511 - val_loss: 1.0436 - val_accuracy: 0.5585

In [ ]:
# Save the model architecture as a JSON file
model_json = model.to_json()
with open('/content/drive/MyDrive/Thesis New/model.json', 'w') as json_file:
    json_file.write(model_json)

# Save the model weights as an HDF5 file
model.save_weights('/content/drive/MyDrive/Thesis New/model.h5')
